# Python에서 MySQL 사용

In [ ]:
!pip install pymysql > /dev/null

In [ ]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

In [ ]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

In [ ]:
import pymysql
conn = pymysql.connect(
    host = config['host']
    user = config['user']
    password = config['password']
    database = config['database']
    port = config['port']
)

In [ ]:
config['database'] = 'mcdb'

## 테이블 생성

In [ ]:
cur = conn.cursor()
sql_user = '''
CREATE TABLE if NOT EXISTS users(
    uid VARCHAR(20) NOT NULL PRIMARY KEY,
    pwd CHAR(44) NOT NULL,
    uname VARCHAR(20) NOT NULL,
    reg_date DATETIME DEFAULT CURRENT_TIMESTAMP,
    is_deleted INT DEFAULT 0
);
'''
cur.execute(sql_user)

## 테이블 구조/이름 변경

In [ ]:
cur.execute('ALTER TABLE users ADD COLUMN email VARCHAR(40)')

## 테이블 삭제

In [ ]:
cur.execute('DROP TABLE users')

## 데이터 삽입

### 기본 스트링 쿼리

In [ ]:
cur = conn.cursor()
cur.execute("INSERT INTO users(uid, uname) VALUES('admin', '관리자');")
cur.execute("INSERT INTO users(uid, uname) VALUES('', ''), ('', ''), ('', '');")
conn.commit()

### 파라미터 : 튜플 사용

In [ ]:
uid = ''
uname = ''
cur = conn.cursor()
sql = 'INSERT INTO users(uid, uname) VALUES(%s, %s);'
cur.execute(sql, users)

### 튜플 리스트 사용

In [ ]:
users = (('', ''), ('', ''), ('', ''))
cur = conn.cursor()
sql = 'INSERT INTO users(uid, uname) VALUES(%s, %s);'
cur.executemany(sql, users)

## 데이터 조회

In [ ]:
sql = """
SELECT uid, uname, DATE_FORMAT(
    reg_date, '%Y-%m-%d %H:%i:%s') AS reg_date
    FROM users WHERE isDeleted=0;
"""

### 순회 조회

In [ ]:
cur = conn.cursor()
cur.execute(sql)
for row in cur:
    print(row)

### 단건 조회

In [ ]:
cur = conn.cursor()
cur.execute(sql)
row = cur.fetchone()

### 다건 조회

In [ ]:
rows = cur.fetchmany(2)

### 모두 조회

In [ ]:
rows = cur.fetchall()
for row in rows:
    print(row)

## 데이터 검색

### 기본 스트링 쿼리

In [ ]:
cur = conn.cursor()
cur.execute("SELECT uname FROM users WHERE uid='';")
rows = cur.fetchall();
for row in rows:
    print(row)

### Place Holder

In [ ]:
cur = conn.cursor()
uid = ''
cur.execute('SELECT uname FROM users WHERE uid=%s;', (uid,))
writer = cur.fetchone()
print(writer[0])

## 종료

In [ ]:
cur.close()
conn.close()